In [1]:
import pandas as pd
import psycopg2 as psy
from sqlalchemy import create_engine
import re
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
with psy.connect(
   host=os.getenv("DB_HOST"),
   port=os.getenv("DB_PORT"),
   database=os.getenv("DB_NAME"),
   user=os.getenv("DB_USER"),
   password=os.getenv("DB_PASSWORD")) as conn:
   cur = conn.cursor()
   cur.execute("SELECT version();")
   print(cur.fetchone())


('PostgreSQL 18.0 on x86_64-windows, compiled by msvc-19.44.35217, 64-bit',)


In [3]:
engine = os.getenv("DB_ENGINE")
connection_string = create_engine(engine)

In [4]:
os.makedirs("../queries_results", exist_ok=True)

In [ ]:
# Quantos pedidos foram feitos por cada cliente?
query1 = """
SELECT 
  c.id as "ID Cliente",
  po.quantity as "Quantidade",
  o.id as "ID do pedido"
  from client c
  join "order" o on c.id = o.client_id
  join product_order po on po.order_id = o.id
  order by c.id;
"""
df1 = pd.read_sql_query(query1, connection_string)
df1.to_excel("../queries_results/qtde_pedidos_por_cliente.xlsx", index=False)

In [11]:
# Algum vendedor também é fornecedor?
query2 = '''
   SELECT
      ss.social_name AS "Razão Social",
      ss.type
      FROM seller_supplier ss
      GROUP BY ss.social_name, ss.type
      HAVING type = 'BOTH';
      '''
df2 = pd.read_sql_query(query2, connection_string)
df2.to_excel('../queries_results/is_vendedor_fornecedor.xlsx')

In [15]:
# Produtos acima de 100 reais
query3 = '''
   SELECT
      p.id AS "ID do produto",
      p.name AS "Nome do produto",
      p.price AS "Preço"
      FROM product p
      WHERE p.price > 100
      ORDER BY p.price DESC;
'''
df3 = pd.read_sql_query(query3, connection_string)
df3.to_excel('../queries_results/produtos_acima_de_100_reais.xlsx')


In [23]:
# Relação nome e preço dos produtos e nome dos fornecedores
query4 = '''
   SELECT
      p.name AS "Nome do Produto",
      p.price AS "Preço do Produto",
      ss.social_name AS "Razão Social",
      pss.seller_supplier_id,
      pss.product_id
      FROM product_seller_supplier pss
      JOIN product p ON p.id = pss.product_id
      JOIN seller_supplier ss ON ss.id = pss.seller_supplier_id
      WHERE ss.type IN ('SUPPLIER', 'BOTH')
      GROUP BY p.name,p.price, ss.social_name, pss.seller_supplier_id, pss.product_id
      ORDER BY p.name, ss.social_name;
'''

df4 = pd.read_sql_query(query4, connection_string)
df4.to_excel('../queries_results/produtos_fornecedores.xlsx', index=False)
